In [1]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.9.1
Uninstalling tensorflow-2.9.1:
  Successfully uninstalled tensorflow-2.9.1


In [3]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [5]:
from bert_score import score
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from evaluate import load
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, StoppingCriteria, StoppingCriteriaList
from transformers import pipeline

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [ ]:
nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

In [ ]:
tokenizer(["a", "b"]).encodings[0].tokens

In [ ]:
print([tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token])

In [ ]:
with torch.no_grad():
    o = model(**tokenizer(["def max(a,b): if a>b: return a else return b", "def min(i,j): if i>j: return i else return j"], return_tensors='pt'))

In [1]:
title_author = "In Search of Lost Time - Marcel Proust"

In [9]:
a,b = title_author.split("-", 1)

In [10]:
a

'In Search of Lost Time '

In [5]:
1 if title_author else None, None

(1, None)

In [ ]:
cos_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [ ]:
o.pooler_output.shape

In [ ]:
o.pooler_output[0,:].unsqueeze(0).shape

In [ ]:
df[df['language_name'] == 'Python'].sample()['code']

In [ ]:
df[df['language_name'] == 'Python']['code'].apply(len).describe()

In [ ]:
tokenizer("\t").encodings[0].tokens

In [ ]:
print(c)

In [ ]:
cos_sim(o.pooler_output[0,:].unsqueeze(0), o.pooler_output[1,:].unsqueeze(0))

In [ ]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def max(i,j): if i>j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

In [ ]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def min(i,j): if i<j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

In [ ]:
print(f"{Precision.numpy()=}, {Recall=}, {F1=}")

In [ ]:
Precision.numpy()

In [ ]:
bloom = AutoModel.from_pretrained('bigscience/bloom-350m')

In [ ]:
tokenizer(["def hello_world():\n", "def add(a,b):"], return_tensors='pt', padding=True)

In [ ]:
g = model.generate(**tokenizer("def hello_world():\n", return_tensors='pt' ,padding=True), 
                   do_sample=True, top_p=0.95, temperature=0.7, num_return_sequences=100)

In [8]:
model = model.to(device)

In [ ]:
!nvidia-smi

In [41]:
g

tensor([[  318, 16509,    63,  ...,  7617,    63,  1045],
        [  318, 16509,    63,  ...,  7617,    63,  1045],
        [  318, 16509,    63,  ...,  7617,    63,  1045],
        ...,
        [  318, 16509,    63,  ...,  7617,    63,  1045],
        [  318, 16509,    63,  ...,  7617,    63,  1045],
        [  318, 16509,    63,  ...,  7617,    63,  1045]], device='cuda:0')

In [ ]:
print(tokenizer.batch_decode(g, skip_special_tokens=True))

In [ ]:
generated_text[len(prompt):].split('\n')

In [6]:
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")
human_eval = load_dataset("openai_humaneval", split='test').filter(lambda x: len(x['canonical_solution'].split('\n'))<3)

NameError: name 'AutoTokenizer' is not defined

'\n\ndef truncate_number(number: float) -> float:\n    """ Given a positive floating point number, it can be decomposed into\n    and integer part (largest integer smaller than given number) and decimals\n    (leftover part always smaller than 1).\n\n    Return the decimal part of the number.\n    >>> truncate_number(3.5)\n    0.5\n    """\n'

In [25]:
model.generate?

In [60]:
%%time
with torch.no_grad():
    g = model.generate(**tokenizer(human_eval[1]['prompt'], return_tensors='pt' ,padding=True).to(device),
                       do_sample=True, typical_p=0.4, temperature=0.6, max_length=300, num_return_sequences=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 2.99 s, sys: 840 ms, total: 3.83 s
Wall time: 3.83 s


In [61]:
g

tensor([[  504, 14090,   492,  ...,   272,   327,  1361],
        [  504, 14090,   492,  ...,   467, 20597,    26],
        [  504, 14090,   492,  ...,   991,    63,   954],
        ...,
        [  504, 14090,   492,  ...,   376,  1324,   769],
        [  504, 14090,   492,  ...,     8,  5465,    26],
        [  504, 14090,   492,  ...,  9388,   376,  1324]], device='cuda:0')

In [62]:
solutions = tokenizer.batch_decode(g, skip_special_tokens=True)

In [64]:
human_eval[1]

{'task_id': 'HumanEval/7',
 'prompt': 'from typing import List\n\n\ndef filter_by_substring(strings: List[str], substring: str) -> List[str]:\n    """ Filter an input list of strings only for ones that contain given substring\n    >>> filter_by_substring([], \'a\')\n    []\n    >>> filter_by_substring([\'abc\', \'bacd\', \'cde\', \'array\'], \'a\')\n    [\'abc\', \'bacd\', \'array\']\n    """\n',
 'canonical_solution': '    return [x for x in strings if substring in x]\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([], 'john') == []\n    assert candidate(['xxx', 'asd', 'xxy', 'john doe', 'xxxAAA', 'xxx'], 'xxx') == ['xxx', 'xxxAAA', 'xxx']\n    assert candidate(['xxx', 'asd', 'aaaxxy', 'john doe', 'xxxAAA', 'xxx'], 'xx') == ['xxx', 'aaaxxy', 'xxxAAA', 'xxx']\n    assert candidate(['grunt', 'trumpet', 'prune', 'gruesome'], 'run') == ['grunt', 'prune']\n",
 'entry_point': 'filter_by_substring'}

In [65]:
print(solutions[10])

from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """
#    print(strings)
    return [s for s in strings if substring in s]


def filter_by_substring_or_empty(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring_or_empty([], 'a')
    []
    >>> filter_by_substring_or_empty(['abc', 'bacd', 'cde', 'array'], 'a')
    []
    """
#    print(strings)
    return [s for s in strings if substring in s]


def filter_by_substring_or_empty_or_none(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring_or_empty_or_non

In [ ]:
human_eval.column_names

In [ ]:
human_eval[10]['canonical_solution']

In [ ]:
df = human_eval.to_pandas()

In [15]:
human_eval

Parameter 'function'=<function <lambda> at 0x7f05e84acaf0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
    num_rows: 37
})

In [ ]:
len(df[df['canonical_solution'].str.split('\n').apply(len) == 3])

In [ ]:
human_eval = load_dataset("openai_humaneval")
code_eval_metric = load("code_eval")

In [ ]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [ ]:
%%time
test_cases = ["assert add(2,3)==5"]
candidates = [["def add(a,b): return a*b", "def add(a, b):\n    return a+b"]]
pass_at_k, results = code_eval_metric.compute(references=test_cases, predictions=candidates, k=[1, 2])
print(pass_at_k)

In [ ]:
print("def add(a, b):\nreturn a+b")

In [ ]:
human_eval['test']

In [ ]:
rosetta = pd.read_json('../code_datasets/code_crawl.jsonl', lines=True)

In [ ]:
rosetta

In [ ]:
from datasets import Dataset

In [ ]:
d = Dataset.from_pandas(rosetta)

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
d.push_to_hub('rosetta-code')

In [ ]:
rosetta = load_dataset('cakiki/rosetta-code', split='train')

In [ ]:
r = rosetta.to_pandas()

In [ ]:
rosetta_python = rosetta.filter(lambda x: x['language_name']=='Python')

In [ ]:
rosetta_python.filter(lambda x: len(x['code'].split('\n')) <= 3 )[:]

In [ ]:
r[r['language_name'] == 'Python']['code'].str.split('\n').apply(len).value_counts().sort_index()[:10]